# The spatial $\Lambda$-Viot-Fleming Process

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cProfile
from scipy.spatial import cKDTree
import time
from slfv import ball_update_allele_distr, rectangle_update_allele_distribution, improved_update_allele_distribution, _get_indices

setting up the parameters

In [ ]:
dim = 2
length = 1
lambda_param = 0.1
no_alleles = 2
discretization_steps = 100
event_rate = 1

Initialization of the simulation

In [ ]:
def initialization(T, dim, length, no_alleles, discretization_steps):
    # define a meshgrid on the area [0, length] ** dim. Every point in the meshgrid needs to hold no_alleles values
    allele_distr = 1 / no_alleles * np.ones((discretization_steps, discretization_steps, no_alleles))

    # simulate the centre of reproduction events
    z = np.random.uniform(0, length, (dim, T))

    # simulate the radius of reproduction events
    r = length / 10 * np.ones(T)

    # simulate the impact of reproduction events
    u = 0.01 * np.ones(T)
    
    return allele_distr, z, r, u

### Control: plot initial gene distribution & centers of reproduction events

In [ ]:
allele_distr, z, r, u = initialization(1000, dim, length, no_alleles, discretization_steps)

plt.title("Initial allele distribution")
plt.imshow(allele_distr[:, :, 0])
plt.colorbar()
plt.show()

plt.title("Center of reproduction events")
plt.scatter(z[0], z[1])
plt.xlim(0, length)
plt.ylim(0, length)
plt.show()

Lets start with the very basic algorithm, implemented in exactly the same straight-forward way as in the paper:

In [ ]:
T = 10 ** 3
allele_distr, z, r, u = initialization(T, dim, length, no_alleles, discretization_steps)
ball_update_allele_distr(allele_distr, z, r, u, T, no_alleles, discretization_steps, length, True)

In [ ]:
T = 10 ** 3
allele_distr, z, r, u = initialization(T, dim, length, no_alleles, discretization_steps)
rectangle_update_allele_distribution(allele_distr, z, r, u, T, no_alleles, discretization_steps, length, True)

In [ ]:
T = 10 ** 3
allele_distr, z, r, u = initialization(T, dim, length, no_alleles, discretization_steps)
improved_update_allele_distribution(allele_distr, z, r, u, T, no_alleles, discretization_steps, length, True)

Comparing both regular and improved algorithm in terms of runtime

In [ ]:
regular = []
improved = []
rectangle = []

for n in range(5):
    T = 10 ** n
    
    allele_distr, z, r, u = initialization(T, dim, length, no_alleles, discretization_steps)
    # measure time
    start = time.time()
    ball_update_allele_distr(allele_distr, z, r, u, T, no_alleles, discretization_steps, length)
    end = time.time()
    regular.append(end-start)

    allele_distr, z, r, u = initialization(T, dim, length, no_alleles, discretization_steps)
    # measure time
    start = time.time()
    improved_update_allele_distribution(allele_distr, z, r, u, T, no_alleles, discretization_steps, length)
    end = time.time()
    improved.append(end-start)

    allele_distr, z, r, u = initialization(T, dim, length, no_alleles, discretization_steps)
    start = time.time()
    rectangle_update_allele_distribution(allele_distr, z, r, u, T, no_alleles, discretization_steps, length)
    end = time.time()
    rectangle.append(end-start)

In [ ]:
plt.plot(regular, label='regular')
plt.plot(improved, label='improved')
plt.plot(rectangle, label='rectangle')
plt.legend()
plt.loglog()
plt.show()

In [ ]:
plt.plot(regular, label='regular')
plt.plot(improved, label='improved')
plt.legend()
plt.show()

In [ ]:
plt.plot(regular, label='regular')
plt.plot(improved, label='improved')
plt.plot(rectangle, label='rectangle')
plt.legend()
plt.yscale('log')
plt.xticks([0, 1, 2, 3, 4], ['10^0', '10^1', '10^2', '10^3', '10^4'])
plt.xlabel('number of events log(T)')
plt.ylabel('run time in seconds log(s)')
plt.show()